# PLY Combination Script

In [1]:
from plyfile import PlyData, PlyElement
import numpy as np
import os

output_vertices = []
traj_data = PlyData.read(os.path.join("data", "KITTI_raw_trajectories", "KITTI_raw_00.PLY"))

In [2]:
traj_data

PlyData((PlyElement('pose', (PlyProperty('qx', 'double'), PlyProperty('qy', 'double'), PlyProperty('qz', 'double'), PlyProperty('qw', 'double'), PlyProperty('x', 'double'), PlyProperty('y', 'double'), PlyProperty('z', 'double'), PlyProperty('ref_t', 'double'), PlyProperty('dest_t', 'double'), PlyProperty('ref_fid', 'int'), PlyProperty('dest_fid', 'int')), count=4540, comments=[]),), text=False, byte_order='<', comments=[], obj_info=[])

In [3]:
def get_rotation_matrix(frame_n):
    global traj_data, previous_rotation_matrix
    qx = traj_data["pose"]["qx"][frame_n]
    qy = traj_data["pose"]["qy"][frame_n]
    qz = traj_data["pose"]["qz"][frame_n]
    qw = traj_data["pose"]["qw"][frame_n]

    return quaternion_rotation_matrix((qx, qy, qz, qw))

def rotate(frame_n, p):
    rm = get_rotation_matrix(frame_n)
    p = p.reshape(3,  -1)
    
    return np.matmul(rm, p).reshape(-1)

def translate(frame_n, p):
    global traj_data
    x = traj_data["pose"]["x"][frame_n]
    y = traj_data["pose"]["y"][frame_n]
    z = traj_data["pose"]["z"][frame_n]

    return np.array([p[0] + x, p[1] + y, p[2] + z])

def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    qx = Q[0]
    qy = Q[1]
    qz = Q[2]
    qw = Q[3]
     
    # First row of the rotation matrix
    r00 = 1 - 2*qy**2 - 2*qz**2
    r01 = 2*qx*qy - 2*qz*qw
    r02 = 2*qx*qz + 2*qy*qw
     
    # Second row of the rotation matrix
    r10 = 2*qx*qy + 2*qz*qw
    r11 = 1 - 2*qx**2 - 2*qz**2
    r12 = 2*qy*qz - 2*qx*qw
     
    # Third row of the rotation matrix
    r20 = 2*qx*qz - 2*qy*qw
    r21 = 2*qy*qz + 2*qx*qw
    r22 = 1 - 2*qx**2 - 2*qy**2
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

def process_frame(frame_n, frame_data: PlyData):
    for x, y, z in zip(frame_data["vertex"]["x"],  frame_data["vertex"]["y"], frame_data["vertex"]["z"]):
            p = np.array([x, y, z])
            output_vertices.append(translate(frame_n, rotate(frame_n, p)))

In [4]:
frame_path = os.path.join("data", "KITTI_raw", "00", "frames")
skip_length = 100
for frame_n, filename in enumerate(os.listdir(frame_path)):
    print(frame_n, filename)
    if frame_n % skip_length == 0:
        print("processing...") 
        frame_data = PlyData.read(os.path.join(frame_path, filename))
        process_frame(frame_n, frame_data)
        
frame_data["vertex"]

0 frame_0000.ply
processing...
1 frame_0001.ply
2 frame_0002.ply
3 frame_0003.ply
4 frame_0004.ply
5 frame_0005.ply
6 frame_0006.ply
7 frame_0007.ply
8 frame_0008.ply
9 frame_0009.ply
10 frame_0010.ply
11 frame_0011.ply
12 frame_0012.ply
13 frame_0013.ply
14 frame_0014.ply
15 frame_0015.ply
16 frame_0016.ply
17 frame_0017.ply
18 frame_0018.ply
19 frame_0019.ply
20 frame_0020.ply
21 frame_0021.ply
22 frame_0022.ply
23 frame_0023.ply
24 frame_0024.ply
25 frame_0025.ply
26 frame_0026.ply
27 frame_0027.ply
28 frame_0028.ply
29 frame_0029.ply
30 frame_0030.ply
31 frame_0031.ply
32 frame_0032.ply
33 frame_0033.ply
34 frame_0034.ply
35 frame_0035.ply
36 frame_0036.ply
37 frame_0037.ply
38 frame_0038.ply
39 frame_0039.ply
40 frame_0040.ply
41 frame_0041.ply
42 frame_0042.ply
43 frame_0043.ply
44 frame_0044.ply
45 frame_0045.ply
46 frame_0046.ply
47 frame_0047.ply
48 frame_0048.ply
49 frame_0049.ply
50 frame_0050.ply
51 frame_0051.ply
52 frame_0052.ply
53 frame_0053.ply
54 frame_0054.ply
55 fra

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('timestamp', 'float')), count=122378, comments=[])

In [5]:
for i, vertex in enumerate(output_vertices):
    output_vertices[i] = tuple(x for x in vertex)

In [6]:
el = PlyElement.describe(np.array(output_vertices, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')]), "vertex")
PlyData([el], text=True).write('test.ply')